In [1]:
import pandas as pd

import numpy as np

import pymongo

import sys

/home/johni/.pyenv/versions/iara-ws-cnmp/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
MONGODB_URL = "mongodb://35.237.232.73:27017/"

client = pymongo.MongoClient(MONGODB_URL)

In [3]:
database_name = "justica_estadual"

collection_name = "processos-tjsp"

processos = list(client[database_name][collection_name].find(limit=100))

print(f"Total de processos {len(processos)}")

Total de processos 100


In [4]:
class Base:
    def __init__(self, filename):
        self.df = pd.read_csv(filename, sep=';')
    
    @property
    def shape(self):
        return self.df.shape
    
    def head(self, n=5):
        return self.df.head(n)
    
    def _buscar(self, codigo):
        df_f = self.df[self.df["codigo"] == codigo]

        if len(df_f) != 1:
            print(f"Código {codigo} não encontrado")
            return ""

        return df_f["descricao"].values[0]

# Classes

In [5]:
class Classe(Base):
    def __init__(self):
        Base.__init__(self, "sgt_classes.csv")

    def processar(self, documentos):
        for documento in documentos:
            classe_processual = documento["dadosBasicos"]["classeProcessual"]
            descricao = self._buscar(classe_processual)
            self._salvar(documento, descricao)

    def _salvar(self, documento, descricao):
        collection = client[database_name][collection_name]

        _id = documento["_id"]

        collection.update_one(
            {
                "_id": _id 
            },
            {
                '$set': { 'dadosBasicos.descricaoClasseProcessual': descricao }
            }
        )

In [6]:
classe = Classe()
classe.shape

(788, 5)

In [7]:
classe.head()

,codigo,descricao,sigla,cod_pai,cod_filhos
0,2,PROCESSO CÍVEL E DO TRABALHO,ProcCivTrab,NaN,"158,175,197,214,1106,12133,159,165,169,1068,11..."
1,5,SUPERIOR TRIBUNAL DE JUSTIÇA,STJ,NaN,"1013,1015,1016,1017,1018,1019,1020,1021,1022,1..."
2,7,Procedimento Comum Cível,ProceComCiv,1107.0,NaN
3,22,Procedimento Sumário,ProSum,1107.0,NaN
4,26,Procedimentos Especiais,ProEsp,1107.0,"27,50,62,28,29,30,31,32,34,35,37,38,39,40,41,4..."


In [8]:
classe.processar(processos)

# Assuntos

In [9]:
class Assuntos(Base):
    def __init__(self):
        Base.__init__(self, "sgt_assuntos.csv")


    def processar(self, documentos):
        for documento in documentos:
            assuntos = documento["dadosBasicos"]["assunto"]
            for indice, assunto in enumerate(assuntos):
                if "codigoNacional" in assunto.keys():
                    codigo_nacional = assunto["codigoNacional"]
                    descricao = self._buscar(codigo_nacional)
                else:
                    descricao = assunto["assuntoLocal"]["descricao"]
            
            self._salvar(documento, indice, descricao)


    def _salvar(self, documento, index, descricao):
    
        collection = client[database_name][collection_name]

        _id = documento["_id"]

        collection.update_one(
            {
                "_id": _id 
            },
            {
                "$set": { f"dadosBasicos.assunto.{index}.descricao": descricao }
            }
        )

In [10]:
assuntos = Assuntos()
assuntos.shape

(3685, 4)

In [11]:
assuntos.head()

,codigo,descricao,cod_pai,cod_filhos
0,14,DIREITO TRIBUTÁRIO,NaN,"5913,5916,5956,5973,5978,5986,6017,6018,6021,6..."
1,195,DIREITO PREVIDENCIÁRIO,NaN,"6094,6119,6160,6173,6181,6095,6096,6099,6100,6..."
2,287,DIREITO PENAL,NaN,"3369,3385,3388,3394,3400,3405,3407,3412,3415,3..."
3,864,DIREITO DO TRABALHO,NaN,"1654,1658,1695,1697,1937,2135,2567,2581,2620,2..."
4,899,DIREITO CIVIL,NaN,"5626,5754,7673,7681,7947,9616,9981,10431,10432..."


In [12]:
assuntos.processar(processos)

# Movimentações

In [13]:
class Movimentacoes(Base):
    def __init__(self):
        Base.__init__(self, "sgt_movimentos.csv")

    def processar(self, documentos):
        for documento in documentos:
            movimentos = documento["movimento"]
            for indice, movimento in enumerate(movimentos):
                if "movimentoNacional" in movimento.keys():
                    codigo = movimento["movimentoNacional"]["codigoNacional"]
                    descricao = self._buscar(codigo)
                    self._salvar(documento, indice, descricao)
                else:
                    print("Código do movimento nacional não encontrado", movimento.keys())                
    
    def _salvar(self, documento, index, descricao):
        collection = client[database_name][collection_name]

        _id = documento["_id"]

        collection.update_one(
            {
                "_id": _id 
            },
            {
                "$set": { f"movimento.{index}.movimentoNacional.descricao": descricao }
            }
        )

In [14]:
movimentacoes = Movimentacoes()
movimentacoes.shape

(693, 4)

In [15]:
movimentacoes.head()

,codigo,descricao,cod_pai,cod_filhos
0,1,Magistrado,NaN,"3,193,11009,7,11,25,56,63,83,108,113,117,122,1..."
1,3,Decisão,1.0,"7,11,25,56,63,83,108,113,117,122,133,138,146,1..."
2,7,Conversão,3.0,"266,821,12140,12473"
3,11,Declaração,3.0,"269,941,1003,12150,12151"
4,14,Serventuário,NaN,"15,18,48,104,865,12522,16,979,980,478,479,480,..."


In [16]:
movimentacoes.processar(processos)

Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 

Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 

Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 

Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 

Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 

...

Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 

Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 

Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 

Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 'orgaoJulgador', 'dataHora'])
Código do movimento nacional não encontrado dict_keys(['movimentoLocal', 

TypeError: 'NoneType' object is not subscriptable